In [ ]:
# install packages
!pip install librosa
!pip install pywt
!pip install matplotlib

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
import pywt

In [ ]:
# target audio file
flight_train_path = '../audios/Barn Owl Nestling\'s Adorable Flight Training Session!.mp3'
y_ft, sr_ft = librosa.load(flight_train_path)

In [ ]:
def get_high_pass_filter(cutoff_freq, sr):
    nyquist = 0.5 * sr
    normal_cutoff = cutoff_freq / nyquist
    return scipy.signal.butter(1, normal_cutoff, btype='high', analog=False)

def produce_spectrogram(y, sr, cutoff_freq, method='stft', amplitude_t=-40, dur=20):
    if method == 'stft':
        # Design a high-pass filter
        b, a = get_high_pass_filter(cutoff_freq, sr)

        y_filtered = scipy.signal.filtfilt(b, a, y)
        if dur != -1:
            y_filtered = y_filtered[:dur * sr]
        D_filtered = librosa.stft(y_filtered)
        S_db_filtered = librosa.amplitude_to_db(np.abs(D_filtered), ref=np.max)

        plt.figure(figsize=(12, 8))
        librosa.display.specshow(S_db_filtered, sr=sr, x_axis='time', y_axis='log')

        plt.ylim(512, sr // 2)

        plt.colorbar(format='%+2.0f dB')
        plt.title('Spectrogram (Frequency vs. Time) with High-Pass Filter at 1024Hz')
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.clim(vmin=amplitude_t)  # Set the minimum amplitude to display
        plt.show()

        # max amplitude
        frequencies = librosa.fft_frequencies(sr=sr, n_fft=D_filtered.shape[0])
        times = librosa.frames_to_time(np.arange(S_db_filtered.shape[1]), sr=sr, hop_length=512, n_fft=2048)
        peak_frequencies = frequencies[np.argmax(S_db_filtered, axis=0)]

        plt.figure(figsize=(12, 8))
        plt.plot(times, peak_frequencies, label='Peak Frequency')
        plt.ylim(512, sr // 2)
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.title('Peak Frequency Over Time for First 20 Seconds')
        plt.legend()
        plt.show()
    if method == 'wavelet':
        if dur != -1:
            y = y[:int(dur * sr)]
        scales = np.arange(1, 128)
        
        coef, freq = pywt.cwt(y, scales, 'cmor', sampling_period=1/sr)
        
        plt.figure(figsize=(12, 8))
        plt.imshow(np.abs(coef), extent=[0, dur, scales[-1], scales[0]], cmap='viridis', aspect='auto',
                   vmax=np.abs(coef).max(), vmin=-np.abs(coef).max())
        plt.colorbar(label='Magnitude')
        plt.title('Wavelet Transform (CWT) of the Audio Signal')
        plt.xlabel('Time (s)')
        plt.ylabel('Scale')
        plt.yscale('log')
        plt.ylim(scales[-1], scales[0])
        plt.show()

In [ ]:
def filter_response_plots(y, sr, freq):
    b, a = get_high_pass_filter(freq, sr_ft)
    w, h = scipy.signal.freqz(b, a)

    plt.figure(figsize=(12, 8))

    plt.subplot(2, 1, 1)
    plt.plot(0.5 * sr * w / np.pi, np.abs(h), 'b')
    plt.title('High-Pass Filter Frequency Response')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Gain')
    plt.grid()

    plt.subplot(2, 1, 2)
    plt.plot(0.5 * sr * w / np.pi, np.angle(h), 'b')
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Phase [radians]')
    plt.grid()

    plt.tight_layout()
    plt.show()

In [ ]:
produce_spectrogram(y_ft, sr_ft, 512, method='stft')
filter_response_plots(y_ft, sr_ft, 512)

In [ ]:
produce_spectrogram(y_ft, sr_ft, 1024, method='stft')
filter_response_plots(y_ft, sr_ft, 1024)

In [ ]:
produce_spectrogram(y_ft, sr_ft, 512, method='wavelet')